In [0]:
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/bosz/bosz_5000_full_3/pca'

In [0]:
%matplotlib inline

In [0]:
import sys

# Allow load project as module
sys.path.insert(0, '../../../..')

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [0]:
import matplotlib     
matplotlib.rcParams.update({'font.size': 14})

# Load stellar model grid

In [0]:
from pfsspec.core.grid import ArrayGrid
from pfsspec.stellar.grid import ModelGrid
from pfsspec.stellar.grid.bosz import Bosz
from pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Bosz(pca=True, normalized=True), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

In [0]:
grid.wave

In [0]:
axes = grid.get_axes()
for k in axes.keys():
    print(k, axes[k].values)

In [0]:
grid.grid.eigs.keys()

## Count valid spectra

In [0]:
grid.array_grid.value_indexes['flux'].shape, np.sum(grid.array_grid.value_indexes['flux'])

# Plot

In [0]:
M_H = 0
T_eff = 4500
log_g = 1
a_M = 0
C_M = 0

idx = grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
model = grid.get_model_at((0, 0, 0, 0, 0))
plt.plot(model.wave, model.flux)

In [0]:
model = grid.get_model_at((0, 0, 0, 0, 0), denormalize=True)
plt.plot(model.wave, model.flux)

# Plot eigenvectors

In [0]:
eigv = grid.grid.eigv['flux']
eigv.shape

In [0]:
for i in range(min(10, eigv.shape[-1])):
    plt.figure()
    plt.plot(grid.wave, eigv[:, i])
    plt.grid()

# Plot principal components

In [0]:
s = np.s_[:, :, :, idx[3], idx[4]]

In [0]:
def load_params(name):
    params = grid.array_grid.get_value(name)
    masks = grid.array_grid.value_indexes[name]
        
    return params, masks

In [0]:
def plot_params(params, idx=2, param_idx=0):
    pp = params.shape[idx]
    rr = int(pp / 4 + 0.5)
    f, axs = plt.subplots(rr, 4, figsize=(16, 4 * rr))
    for p in range(pp):
        i = p // 4
        j = p % 4
        
        s = (params.ndim - 1) * [slice(None)]
        s[idx] = p
        s = tuple(s)
        vmin, vmax = params[s][..., param_idx].min(), params[s][..., param_idx].max()
        l = axs[i, j].imshow(params[s][..., param_idx], aspect='auto') #, vmin=vmin, vmax=vmax)
        f.colorbar(l, ax=axs[i, j])
        axs[i, j].set_xlabel('param: {} | slice: {}'.format(param_idx, p))

In [0]:
params, masks = load_params('flux')
params.shape[-1]

In [0]:
for pi in range(min(15, params.shape[-1])):
    plot_params(params[s], param_idx=pi)